<h3> This notebook attempts to read in IQuOD intelligent meta data from Tim Boyer's NetCDF files </h3> 

In [49]:
import netCDF4 as nc4
import numpy as np
import matplotlib.pyplot as plt 
import datetime as dt
import string

In [ ]:
datadir = '/Users/matt/Data/WOD/IQuOD/'
ncfile  = 'iquod_xbt_1987.nc'

f = nc4.Dataset(datadir+ncfile,'r', format='NETCDF4')

print "meta data for the dataset:"
print(f)

In [ ]:
f.variables.keys()



<h4>This code loops through the profiles that contain iMetaData and strips out the information needed to 
cross-check against the IQuOD v0.1 algorithm..</h4> 

In [86]:
def extract_date(date):
    # Input is a int8 date in format, e.g. 19780101
    # The function returns a python date-time object
    datestr = date.astype('str') # Convert to string to chop the date into year, month, day
    yr = datestr[0:4]
    mn = datestr[4:6]
    dy = datestr[6:]
    dateobj = dt.date(int(yr), int(mn), int(dy))
    return dateobj

datadir = '/Users/matt/Data/WOD/IQuOD/'
ncfile  = 'iquod_xbt_1987.nc'

f = nc4.Dataset(datadir+ncfile,'r', format='NETCDF4')

depths = f.variables['z']
depth_sizes = f.variables['z_row_size']
country = f.variables['country']
date = f.variables['date']
instr_name = f.variables['Temperature_Instrument']
instr_imeta = f.variables['Temperature_Instrument_intelligentmetadata'] # Flag for iMetaData (True or False)
WOD_unique = f.variables['wod_unique_cast'] # Unique WOD cast number 
d_index = np.cumsum(depth_sizes)-1 # Create index for maximum depths 
max_depths = depths[d_index] # This seems to work fine - note that there are some negative depth values!! 

mask = instr_imeta[:].data # Determine profiles that contain iMetaData
mindex = np.where(mask == 1)

# Restrict all information to those pertaining to iMetaData

country = country[mindex]
instr_name = instr_name[mindex]
WOD_unique = WOD_unique[mindex]
max_depths = max_depths[mindex]

# Create an empty dictionary to storet the salient information..

imeta_dict = {}

#print 'len(country) = ',len(country)
#print 'len(WOD_unique) = ',len(WOD_unique)

for ii, ww in enumerate(WOD_unique):
#    print ii
    cntry     = string.join(country[ii].data)
    ptype     = string.join(instr_name[ii].data)
    max_depth = max_depths[ii]
    dateobj   = extract_date(date[ii])
    newlist   = [cntry, max_depth, dateobj, ptype]
    imeta_dict.update({ww:newlist})

#WOD_unique_iMeta = WOD_unique[mindex]
#date_iMeta = date_iMeta[mindex]
#instr_name_iMeta = instr_name_iMeta[mindex]

#print WOD_unique_iMeta



In [97]:
pickle.dump( imeta_dict, open( datadir + 'iquod_xbt_1987_iMetaData_summary.pickle', 'wb' ) )

#print imeta_dict

In [109]:
# Read in the save file and extract all unique instrument types..

datadir = '/Users/matt/Data/WOD/IQuOD/'
pfile   = 'iquod_xbt_1987_iMetaData_summary.pickle'

d = pickle.load( open( datadir + pfile, 'rb' ) )

# Initialise an empty list for instr_name
instr_name = []

# Initialise an empty list for country_name
country_name = []

for key, value in d.iteritems():
    country_name.append(value[0])
    instr_name.append(value[3])





In [122]:
instr_name_set = set(instr_name)
country_name_set = set(country_name)

print instr_name_set

ccode = "J A P A N                                   "

if "J A P A N" in ccode:
    man="( T S K   -   T S U R U M I   S E I K I   C o . )"
    
print man

max_depth = 1441.5
pdate = dt.date(1987, 1, 1)

imeta = iquod_v01_imeta(max_depth, pdate, ccode)

print imeta

#print instr_name_set
#print country_name_set

#print list(country_name_set)

set(['X B T :   T 4   ( S I P P I C A N )                                                                                                                                                        ', 'X B T :   T 5   ( S I P P I C A N )                                                                                                                                                        ', 'X B T :   T 5   ( T S K   -   T S U R U M I   S E I K I   C o . )                                                                                                                                         ', 'X B T :   T 7   ( T S K   -   T S U R U M I   S E I K I   C o . )                                                                                                                                         ', 'X B T :   T 6   ( T S K   -   T S U R U M I   S E I K I   C o . )                                                                                                                                       

<h4> Loop over the summary file and write out lists of the iMetaData..</h4>

In [137]:
iMeta_Tim = []
iMeta_Matt = []

for key, value in d.iteritems():
    ccode = value[0]
    max_depth = value[1]
    pdate = value[2]
    iMeta_Tim.append(string.strip(value[3])) # Extract Tim's iMeta value
    iMeta = iquod_v01_imeta(max_depth, pdate, ccode) # Use v0.1 algorithm on profile info
    iMeta_Matt.append(iMeta) # Assign to Matt's iMeta value

    
print iMeta_Tim[0:10]
print iMeta_Matt[0:10]









['X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 6   ( T S K   -   T S U R U M I   S E I K I   C o . )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )']
['X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )', 'X B T :   T 4   ( S I P P I C A N )']
51860
51860


In [141]:
print len(iMeta_Tim)

51860


In [140]:
cc = 0 # Initialise counter
for ii in iMeta_Matt:
    jj = iMeta_Tim[cc]
    if ii != jj:
        print ii
    cc+=1 

X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   T 4   ( T S K   -   T S U R U M I   S E I K I   C o . )
X B T :   

In [138]:
# Now write both summaries out to textfile..

datadir = "/Users/matt/Data/WOD/IQuOD/"
filename1   = "iMeta_1987_Tim.txt"
filename2   = "iMeta_1987_Matt.txt"

file1 = open(datadir+filename1, "w")
for line in iMeta_Tim:
  file1.write("%s\n" % line)
file1.close()


file2 = open(datadir+filename2, "w")
for line in iMeta_Matt:
  file2.write("%s\n" % line)
file2.close()



In [118]:
# Main function - adapted to work with the current output (from NetCDF4 files rather than ASCII)



# Need to update the cutoff depths and dates based on Palmer et al (in prep)..

def iquod_v01_imeta(max_depth, pdate, ccode):
    """
    This function generates intelligent metadata for eXpendable BathyThermographs (XBTs)
    following the approach outlined in Palmer et al (in prep)
    
    
    Input Parameters:
    -----------------
    
    * max_depth = the maximum depth recorded by the profile
    
    * pdate = a datetime object to describe when the profile was recorded
    
    * ccode = the country code (string), which determines the manufacturer
    
    
    Returns:
    --------
    
    * ptype = the probe type as a human-readable string 
    
    """
    
    man="( S I P P I C A N )" # Default manufacturer is Sippican
    
    if "J A P A N" in ccode:
        man="( T S K   -   T S U R U M I   S E I K I   C o . )"

    baddepth = False # Default value for "baddepth" flag
    ptype    = "UNKNOWN" # Default value for prtyp is UNKOWN 
    
    dcut = np.array([360., 600., 1000., 1350., 2000.]) # Depth cut-offs

    tcut_sipp = {"T10":dt.date(1993, 1, 1), "DB":dt.date(1997, 1, 1),
                 "FD":dt.date(2007, 1, 1)}

    tcut_tsk = {"T6":dt.date(1995, 1, 1), "T6_shallow":dt.date(1996, 1, 1),
                "T7":dt.date(1979, 1, 1)}
    
    if max_depth > 2000: # Depth is beyond 2500, probably NOT an XBT..
        baddepth = True 
        
    if baddepth == False: # If max_depth is OK, continue.. 
        
        di = np.where(max_depth <= dcut)[0][0] # "di"  = depth index - only need the first occurence
        if man == "( S I P P I C A N )":
            if di == 0: # If cut-off depth <= 360m assign T4 or a T10
                tdelta = (pdate-tcut_sipp["T10"]).days # Compute the time-delta in days from T10 date-to-market
                if tdelta >= 0.:   
                    ptype = "T 10"
                else:              
                    ptype = "T 4"
            elif di == 1: # If 360m < cut-off depth <= 600m assign T4
                ptype = "T 4"
            elif di == 2: # If 600m < cut-off depth <= 1000m assign T7 or Deep Blue..
                tdelta = (pdate-tcut_sipp["DB"]).days
                if tdelta >= 0.:
                    ptype = "D B"
                else:
                    ptype = "T 7"
            elif di == 3: # If 1000m < cut-off depth <= 1350m assign T5 or Fast Deep
                tdelta = (pdate-tcut_sipp["FD"]).days
                if tdelta >= 0.:
                    ptype = "F D"
                else:
                    ptype = "T 5"                
            else: # If 1350m < cut-off depth <= 2500m assign T5 
                ptype = "T 5"    
        else: # If NOT Sippican, manufacturer must be TSK..            
            if di == 0: # If cut-off depth < 360m assign T4 or T6
                tdelta = (pdate-tcut_tsk["T6_shallow"]).days # Compute the time-delta in days from T10 date-to-market
                if tdelta >= 0.:   
                    ptype = "T 6"
                else:            
                    ptype = "T 4"
            elif di == 1: # If 360m < cut-off depth <= 600m assign T4 or T6 
                tdelta = (pdate-tcut_tsk["T6"]).days # Compute the time-delta in days from T10 date-to-market
                if tdelta >= 0.:   
                    ptype = "T 6"
                else:               
                    ptype = "T 4"             
            elif di == 2: # If 600m < cut-off depth <= 1000m assign T5 or T7 
                tdelta = (pdate-tcut_tsk["T7"]).days
                if tdelta >= 0.:
                    ptype = "T 7"
                else:
                    ptype = "T 5"
            else: # If 1000m < cut-off depth <= 2500m assign T5 
                ptype = "T 5"             
    return "X B T :   "+ptype+"   "+man

In [ ]:
# Main function.. 

# Need to update the cutoff depths and dates based on Palmer et al (in prep)..

def iquod_v01_imeta(max_depth, pdate, ccode):
    """
    This function generates intelligent metadata for eXpendable BathyThermographs (XBTs)
    following the approach outlined in Palmer et al (in prep)
    
    
    Input Parameters:
    -----------------
    
    * max_depth = the maximum depth recorded by the profile
    
    * pdate = a datetime object to describe when the profile was recorded
    
    * ccode = the country code (string), which determines the manufacturer
    
    
    Returns:
    --------
    
    * ptype = the probe type as a human-readable string 
    
    """
    
    man="(SIPPICAN)" # Default manufacturer is Sippican
    
    if ccode in ["JP", "TW", "CN", "KR"]:
        man="(TSK - TSURUMI SEIKI Co.)"

    baddepth = False # Default value for "baddepth" flag
    ptype    = "UNKNOWN" # Default value for prtyp is UNKOWN 
    
    dcut = np.array([360., 600., 1000., 1350., 2500.]) # Depth cut-offs

    tcut_sipp = {"T10":dt.date(1993, 1, 1), "DB":dt.date(1997, 1, 1),
                 "FD":dt.date(2007, 1, 1)}

    tcut_tsk = {"T6":dt.date(1995, 1, 1), "T6_shallow":dt.date(1996, 1, 1)
                "T7":dt.date(1979, 1, 1)}
    
    if max_depth > 2500: # Depth is beyond 2500, probably NOT an XBT..
        baddepth = True 
        
    if baddepth == False: # If max_depth is OK, continue.. 
        
        di = np.where(max_depth <= dcut)[0][0] # "di"  = depth index - only need the first occurence
        if man == "(SIPPICAN)":
            if di == 0: # If cut-off depth <= 360m assign T4 or a T10
                tdelta = (pdate-tcut_sipp["T10"]).days # Compute the time-delta in days from T10 date-to-market
                if tdelta >= 0.:   
                    ptype = "T10"
                else:              
                    ptype = "T4"
            elif di == 1: # If 360m < cut-off depth <= 600m assign T4
                ptype = "T4"
            elif di == 2: # If 600m < cut-off depth <= 1000m assign T7 or Deep Blue..
                tdelta = (pdate-tcut_sipp["DB"]).days
                if tdelta >= 0.:
                    ptype = "DB"
                else:
                    ptype = "T7"
            elif di == 3: # If 1000m < cut-off depth <= 1350m assign T5 or Fast Deep
                tdelta = (pdate-tcut_sipp["FD"]).days
                if tdelta >= 0.:
                    ptype = "FD"
                else:
                    ptype = "T5"                
            else: # If 1350m < cut-off depth <= 2500m assign T5 
                ptype = "T5"    
        else: # If NOT Sippican, manufacturer must be TSK..            
            if di == 0: # If cut-off depth < 360m assign T4 or T6
                tdelta = (pdate-tcut_tsk["T6_shallow"]).days # Compute the time-delta in days from T10 date-to-market
                if tdelta >= 0.:   
                    ptype = "T6"
                else:            
                    ptype = "T4"
            elif di == 1: # If 360m < cut-off depth <= 600m assign T4 or T6 
                tdelta = (pdate-tcut_tsk["T6"]).days # Compute the time-delta in days from T10 date-to-market
                if tdelta >= 0.:   
                    ptype = "T6"
                else:               
                    ptype = "T4"             
            elif di == 2: # If 600m < cut-off depth <= 1000m assign T5 or T7 
                tdelta = (pdate-tcut_tsk["T7"]).days
                if tdelta >= 0.:
                    ptype = "T7"
                else:
                    ptype = "T5"
            else: # If 1000m < cut-off depth <= 2500m assign T5 
                ptype = "T5"             
    return ptype, man